We've included 2 solutions here. There are also many other ways of formatting this data. Don't wory if you haven't done it exactly like we have.... this is just guide and not a bible.

In [1]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Solution using a for loop

First we need to take a look at the JSON so we know what we're dealing with.
- We can view what the JSON contains by checking out the [documentation on the OWM website](https://openweathermap.org/forecast5)
- Or this can be done manually, like below:

In [ ]:
city = 'Berlin'
API_key = 'Your API Key'

# check out the docs for more info on making an api call https://openweathermap.org/forecast5

url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
response = requests.get(url)
json = response.json()
json

Now we've discovered what information we have to work with. Let's decide what we want to keep and what we wish to lose.

I feel that from json['list'] it would be good to keep
- 'weather.main', 'weather.description', 'dt_txt', 'main.temp', 'main.feels_like' 'clouds.all', 'rain.3h', 'snow.3h' 'wind.speed', 'wind.deg', 'main.humidity', 'main.pressure'. 

And from json['city'] it would be good to keep 
- 'name' and 'country. 

Just to make sure that we got the right place. And as an added extra we'll also include the time that API call was made, so we know how up to date our forecast is.

**Optional:** let's get a timestamp of when we get the data. Datetime uses the uses the current time of the system, which on local computers is normally correct. But as we're in the cloud, computers are not always in our country, and we therefore add on the timezone module to ensure that our timestamp is local to us and not the computer.

In [3]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

now

datetime.datetime(2023, 4, 5, 15, 33, 6, 968793, tzinfo=<DstTzInfo 'Europe/Berlin' CEST+2:00:00 DST>)

**Next** let's loop through the json['list'] information get the weather information

In [4]:
# we'll store the information in this dicitonary:
weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []
                }

# let's begin the loop
for i in json['list']:
  weather_dict['city'].append(json['city']['name'])
  weather_dict['country'].append(json['city']['country'])
  weather_dict['forecast_time'].append(i['dt_txt'])
  weather_dict['outlook'].append(i['weather'][0]['main'])
  weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
  weather_dict['temperature'].append(i['main']['temp'])
  weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
  weather_dict['clouds'].append(i['clouds']['all'])
  # sometimes the data is missing for rain and snow. As it is not always raining or snowing
  # we cannot make a DataFrame unless the lists are all the same length, therefore missing values are bad
  # here we say try to append a value if there is one. If not, append a 0
  try:
      weather_dict['rain'].append(i['rain']['3h'])
  except:
      weather_dict['rain'].append('0')
  try:
      weather_dict['snow'].append(i['snow']['3h'])
  except:
      weather_dict['snow'].append('0')
  weather_dict['wind_speed'].append(i['wind']['speed'])
  weather_dict['wind_deg'].append(i['wind']['deg'])
  weather_dict['humidity'].append(i['main']['humidity'])
  weather_dict['pressure'].append(i['main']['pressure'])
  weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
  

**Now** we convert our dictionary to a DataFrame

In [5]:
weather_from_dict_df = pd.DataFrame(weather_dict)

weather_from_dict_df.head()

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-04-05 15:00:00,Rain,light rain,7.65,6.42,83,0.74,0,2.04,332,53,1014,05/04/2023 15:33:06
1,Berlin,DE,2023-04-05 18:00:00,Rain,light rain,6.10,6.10,90,0.48,0,0.71,38,64,1018,05/04/2023 15:33:06
2,Berlin,DE,2023-04-05 21:00:00,Rain,light rain,3.85,2.41,85,0.15,0,1.67,163,76,1022,05/04/2023 15:33:06
3,Berlin,DE,2023-04-06 00:00:00,Clouds,broken clouds,2.16,0.18,60,0,0,1.90,180,82,1022,05/04/2023 15:33:06
4,Berlin,DE,2023-04-06 03:00:00,Clouds,few clouds,1.03,-0.69,20,0,0,1.57,182,82,1022,05/04/2023 15:33:06


**As a final step**, to keep everything tidy: let's bring everything we did together in a function. And allow the function to take a list of cities as an input

In [6]:
def get_weather_loop(cities):

  API_key = 'YOUR_API_KEY_HERE'

  tz = pytz.timezone('Europe/Berlin')
  now = datetime.now().astimezone(tz)

  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    for i in json['list']:
      weather_dict['city'].append(json['city']['name'])
      weather_dict['country'].append(json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      try:
          weather_dict['rain'].append(i['rain']['3h'])
      except:
          weather_dict['rain'].append('0')
      try:
          weather_dict['snow'].append(i['snow']['3h'])
      except:
          weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  return pd.DataFrame(weather_dict)

In [ ]:
get_weather_loop(['Berlin', 'London'])

# Solution using `.json_normalize()`

**First** we need to make an API call to get our information

In [8]:
city = 'Berlin'
API_key = 'Your API Key'
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
response = requests.get(url)
json = response.json()

If we now try to use `.json_normalize()` on the `list` part of the json, you'll see that json_normalize can get confused when it hits a list instead of a subdictionary. Take a look at the weather column below

In [9]:
pd.DataFrame((json['list'])).head()

,dt,main,weather,clouds,wind,visibility,pop,rain,sys,dt_txt
0,1680706800,"{'temp': 8.12, 'feels_like': 6.97, 'temp_min':...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 75},"{'speed': 2.04, 'deg': 332, 'gust': 2.75}",10000,0.58,{'3h': 0.74},{'pod': 'd'},2023-04-05 15:00:00
1,1680717600,"{'temp': 7.15, 'feels_like': 7.15, 'temp_min':...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 83},"{'speed': 0.71, 'deg': 38, 'gust': 0.95}",10000,0.52,{'3h': 0.48},{'pod': 'n'},2023-04-05 18:00:00
2,1680728400,"{'temp': 5.27, 'feels_like': 4.04, 'temp_min':...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 82},"{'speed': 1.67, 'deg': 163, 'gust': 2.62}",10000,0.20,{'3h': 0.15},{'pod': 'n'},2023-04-05 21:00:00
3,1680739200,"{'temp': 2.16, 'feels_like': 0.18, 'temp_min':...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 60},"{'speed': 1.9, 'deg': 180, 'gust': 3.28}",10000,0.00,NaN,{'pod': 'n'},2023-04-06 00:00:00
4,1680750000,"{'temp': 1.03, 'feels_like': -0.69, 'temp_min'...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 20},"{'speed': 1.57, 'deg': 182, 'gust': 2.25}",10000,0.00,NaN,{'pod': 'n'},2023-04-06 03:00:00


To fix this we need to flatten the nested list using the parameter `record_path`, and then select the other columns we'd like using the parameter `meta`. At the end of the json_normalize parameters we have `errors='ignore'`, which means that if ever there is a missing value in the JSON, a `NaN` gets inserted into the table. If you'd like more information, please check out the [docs](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html)

In [10]:
json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']],
                                errors='ignore')

json_norm_df.head()

,id,main,description,icon,dt_txt,main.temp,main.feels_like,clouds.all,rain.3h,snow.3h,wind.speed,wind.deg,main.humidity,main.pressure
0,500,Rain,light rain,10d,2023-04-05 15:00:00,8.12,6.97,75,0.74,NaN,2.04,332,51,1010
1,500,Rain,light rain,10n,2023-04-05 18:00:00,7.15,7.15,83,0.48,NaN,0.71,38,57,1014
2,500,Rain,light rain,10n,2023-04-05 21:00:00,5.27,4.04,82,0.15,NaN,1.67,163,68,1018
3,803,Clouds,broken clouds,04n,2023-04-06 00:00:00,2.16,0.18,60,NaN,NaN,1.9,180,82,1022
4,801,Clouds,few clouds,02n,2023-04-06 03:00:00,1.03,-0.69,20,NaN,NaN,1.57,182,82,1022


Now let's tidy this DataFrame up by
- dropping some columns
- renaming some columns
- adding some columns

In [11]:
# drop id and icon
json_norm_df = json_norm_df.drop(columns=['id', 'icon']).copy()

In [12]:
# rename some columns
json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)

In [13]:
# add some columns
json_norm_df.insert(0, 'city', json['city']['name'])
json_norm_df.insert(1, 'country', json['city']['country'])
json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")

# rearrange the columns
json_norm_df = json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
       'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
       'wind_speed', 'wind_deg', 'humidity', 'pressure',
       'information_retrieved_at']]

json_norm_df.head()

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-04-05 15:00:00,Rain,light rain,8.12,6.97,75,0.74,NaN,2.04,332,51,1010,05/04/2023 15:33:06
1,Berlin,DE,2023-04-05 18:00:00,Rain,light rain,7.15,7.15,83,0.48,NaN,0.71,38,57,1014,05/04/2023 15:33:06
2,Berlin,DE,2023-04-05 21:00:00,Rain,light rain,5.27,4.04,82,0.15,NaN,1.67,163,68,1018,05/04/2023 15:33:06
3,Berlin,DE,2023-04-06 00:00:00,Clouds,broken clouds,2.16,0.18,60,NaN,NaN,1.9,180,82,1022,05/04/2023 15:33:06
4,Berlin,DE,2023-04-06 03:00:00,Clouds,few clouds,1.03,-0.69,20,NaN,NaN,1.57,182,82,1022,05/04/2023 15:33:06


Now we have a very similar DataFrame to what we had with the for loop. Again, let's put this in a function that can take a list of cities

In [14]:
def get_weather_norm(cities):
  
  API_key = 'Your api key'

  df_list = []

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
    response = requests.get(url)
    json = response.json()

    json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']], 
                                errors='ignore')
    json_norm_df.drop(columns=['id', 'icon'], inplace=True)
    json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)
    json_norm_df.insert(0, 'city', json['city']['name'])
    json_norm_df.insert(1, 'country', json['city']['country'])
    json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")
    json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
          'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
          'wind_speed', 'wind_deg', 'humidity', 'pressure',
          'information_retrieved_at']]
    df_list.append(json_norm_df)
  return pd.concat(df_list, ignore_index=True)

In [15]:
get_weather_norm(['Berlin', 'Copenhagen'])

,city,country,outlook,detailed_outlook,forecast_time,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,Rain,light rain,2023-04-05 15:00:00,8.12,6.97,75,0.74,NaN,2.04,332,51,1010,05/04/2023 15:33:06
1,Berlin,DE,Rain,light rain,2023-04-05 18:00:00,7.15,7.15,83,0.48,NaN,0.71,38,57,1014,05/04/2023 15:33:06
2,Berlin,DE,Rain,light rain,2023-04-05 21:00:00,5.27,4.04,82,0.15,NaN,1.67,163,68,1018,05/04/2023 15:33:06
3,Berlin,DE,Clouds,broken clouds,2023-04-06 00:00:00,2.16,0.18,60,NaN,NaN,1.9,180,82,1022,05/04/2023 15:33:06
4,Berlin,DE,Clouds,few clouds,2023-04-06 03:00:00,1.03,-0.69,20,NaN,NaN,1.57,182,82,1022,05/04/2023 15:33:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Copenhagen,DK,Clouds,overcast clouds,2023-04-10 00:00:00,4.06,2.08,100,NaN,NaN,2.19,54,79,1025,05/04/2023 15:33:06
76,Copenhagen,DK,Clouds,overcast clouds,2023-04-10 03:00:00,3.77,1.77,97,NaN,NaN,2.16,55,82,1025,05/04/2023 15:33:06
77,Copenhagen,DK,Clouds,overcast clouds,2023-04-10 06:00:00,4.19,1.86,98,NaN,NaN,2.59,25,86,1025,05/04/2023 15:33:06
78,Copenhagen,DK,Rain,light rain,2023-04-10 09:00:00,4.93,2.6,100,0.2,NaN,2.76,17,89,1025,05/04/2023 15:33:06


In [18]:
get_weather_norm

<function __main__.get_weather_norm(cities)>

# Establishing connection from python to the sql

In [19]:
!pip install pymysql
import sqlalchemy

In [20]:
schema="gans"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="xxxx" # your password!!!!
port=3306
connection_details = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [21]:
get_weather_norm(['Berlin', 'Copenhagen']).to_sql('weather',con=connection_details,if_exists='append',index=False)

80

In [ ]:
#Successfully established the connection from python to the sql